Deep Neural Network in TensorFlow
You've seen how to build a logistic classifier using TensorFlow. Now you're going to see how to use the logistic classifier to build a deep neural network.

Step by Step
In the following walkthrough, we'll step through TensorFlow code written to classify the letters in the MNIST database. If you would like to run the network on your computer, the file is provided here. You can find this and many more examples of TensorFlow at Aymeric Damien's GitHub repository.

In [1]:
### TensorFlow MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
### Learning Parameters

import tensorflow as tf

#Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 128 # Decrease batch size if you don't have enough memory
display_step = 1

n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10


In [3]:
### Hidden Layer Parameters
# The focus here is on the architecture of multilayer neural networks, 
# not parameter tuning, so here we'll just give you the learning parameters.

# The variable `n_hidden_layer` determines the size of the hidden layer in the neural network. 
# This is also known as the width of a layer.
n_hidden_layer = 256 # layer number of features


In [5]:
### Weights and Biases

# Store layers weight and biases
weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}

biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

Deep neural networks use multiple layers with each layer requiring it's own weight and bias. The `hidden_layer` weight and bias is for the hidden layer. The `out` weight and bias is for the output layer. If the neural network were deeper, there would be weights and biases for each additional layer.

In [6]:
### INPUT
# tf Graph input
x = tf.placeholder("float", [None, 28, 28, 1])
y = tf.placeholder("float", [None, n_classes])

# The MNIST data is made up of 28px by 28px images with a single channel. 
# The tf.reshape() function above reshapes the 28px by 28px matrices in x into row vectors of 784px.
x_flat = tf.reshape(x, [-1, n_input])

### Multilayer Perceptron
![](multi-layer.png)

In [7]:
# Hidden layer with RELU activation
layer_1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']), biases['hidden_layer'])
layer_1 = tf.nn.relu(layer_1)

# Output layer with linear activation
logits = tf.add(tf.matmul(layer_1, weights['out']), biases['out'])

You've seen the linear function `tf.add(tf.matmul(x_flat, weights['hidden_layer']), biases['hidden_layer']) `before, also known as `xw + b`. Combining linear functions together using a ReLU will give you a two layer network.

In [8]:
### OPTIMIZER
#define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [10]:
### Session
# Init the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all the batches
        for i in range(total_batch):
            # The MNIST library in TensorFlow provides the ability to receive the dataset in batches. 
            # Calling the mnist.train.next_batch() function returns a subset of the training data.
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        # Display logs per epoch step
        if epoch % display_step == 0:
            c = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(c))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    # Decrease test_size if you don't have enough memory
    test_size = 256
    print("Accuracy:", accuracy.eval({x: mnist.test.images[:test_size], y: mnist.test.labels[:test_size]}))


Epoch: 0001 cost= 37.752876282
Epoch: 0002 cost= 26.031988144
Epoch: 0003 cost= 16.877628326
Epoch: 0004 cost= 14.873718262
Epoch: 0005 cost= 13.405359268
Epoch: 0006 cost= 14.847957611
Epoch: 0007 cost= 10.604688644
Epoch: 0008 cost= 10.079156876
Epoch: 0009 cost= 14.232748985
Epoch: 0010 cost= 7.666872978
Epoch: 0011 cost= 14.318511009
Epoch: 0012 cost= 6.023668766
Epoch: 0013 cost= 8.966882706
Epoch: 0014 cost= 9.073411942
Epoch: 0015 cost= 9.403985023
Epoch: 0016 cost= 4.259663105
Epoch: 0017 cost= 7.344569683
Epoch: 0018 cost= 5.804015160
Epoch: 0019 cost= 7.474333286
Epoch: 0020 cost= 4.746312141
Optimization Finished!
Accuracy: 0.828125
